In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
# Note: Since the same api_keys file is used for multiple activities, it should be in the 
# folder one level below this notebook
import sys
sys.path.append('../')
from api_keys import g_key


In [2]:
city_data = pd.read_csv("../output_data/citydata.csv")
city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hamilton,39.1834,-84.5333,73.96,47,1,1.01,US,1634669631
1,Dikson,73.5069,80.5464,27.82,57,100,11.86,RU,1634669156
2,Sālūmbar,24.1333,74.0500,75.33,53,100,1.70,IN,1634669632
3,Rikitea,-23.1203,-134.9692,74.93,71,0,20.42,PF,1634669090
4,Punta Arenas,-53.1500,-70.9167,62.71,51,75,35.68,CL,1634669094
...,...,...,...,...,...,...,...,...,...
536,Laredo,27.5064,-99.5075,87.80,45,100,15.64,US,1634670052
537,Diamantino,-14.4086,-56.4461,85.10,55,93,2.95,BR,1634670065
538,Walvis Bay,-22.9575,14.5053,59.09,85,100,7.49,NaN,1634670066
539,Saint-André-Avellin,45.7168,-75.0660,61.61,58,100,10.38,CA,1634670066


In [3]:
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = city_data[["Lat", "Lng"]]

# Fill NaN values and convert to float
rating = city_data["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
#Start narrowing cities by weather preferences
#Drop cities with max temp > 85
cities_myweather = city_data[(city_data["Max Temp"] <= 85)]
#Drop cities with max temp < 50
cities_myweather = cities_myweather[(cities_myweather["Max Temp"] >= 50)]
#Drop cities with humidity > 70
cities_myweather = cities_myweather[(cities_myweather["Humidity"] <= 70)]
#Drop cities with humidity < 40 (Not TOO Dry...No deserts)
cities_myweather = cities_myweather[(cities_myweather["Humidity"] >= 40)]
#Drop US cities since I want to travel international
cities_myweather = cities_myweather[(cities_myweather["Country"] != "US")]
cities_myweather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,Sālūmbar,24.1333,74.0500,75.33,53,100,1.70,IN,1634669632
4,Punta Arenas,-53.1500,-70.9167,62.71,51,75,35.68,CL,1634669094
7,Taltal,-25.4000,-70.4833,63.23,64,27,4.99,CL,1634669634
10,Ribeira Grande,38.5167,-28.7000,72.00,68,40,10.36,PT,1634669636
11,Constitución,-35.3333,-72.4167,65.12,59,16,6.33,CL,1634669637
...,...,...,...,...,...,...,...,...,...
514,Biu,10.6129,12.1946,75.99,66,64,2.71,NG,1634670049
518,Mzimba,-11.9000,33.6000,64.09,65,1,6.24,MW,1634670052
522,Imbituba,-28.2400,-48.6703,65.37,63,100,8.50,BR,1634670055
533,Natal,-5.7950,-35.2094,82.35,63,81,12.59,BR,1634670062


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = cities_myweather.copy()
hotel_df["Hotel Name"] = ""
hotel_df = hotel_df.reset_index()
hotel_df



# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


for index in range(len(hotel_df)):
    #target_coordinates 
    target_coordinates = f"{hotel_df.loc[index, 'Lat']}, {hotel_df.loc[index, 'Lng']}"
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Processing Record {index} : {results[0]['name']}")
        hotel_df.loc[index, 'Lat'] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Lng'] = results[0]['geometry']['location']['lng']
        hotel_df.loc[index, 'Hotel Name'] = results[1]['name']
        
    except (KeyError, IndexError):
        print(f"Missing result... skipping {hotel_df.loc[index, 'City']}")
        

        
#Drop Cities with no hotels 
hotel_df = hotel_df[(hotel_df["Hotel Name"] != "")]
        
hotel_df


Processing Record 0 : chatra chaya guest house
Processing Record 1 : Hotel Dreams Del Estrecho
Processing Record 2 : Residencial Karime
Processing Record 3 : Quinta da Meia Eira
Processing Record 4 : Hotel Boutique Las Azucenas
Processing Record 5 : Albatros Hotel
Processing Record 6 : خوابگاه اساتيد دانشگاه دامغان
Processing Record 7 : Diego De Almagro Coyhaique
Processing Record 8 : Moon light Furnished Units
Processing Record 9 : Abaco Beach Resort
Processing Record 10 : Southern Sun Waterfront Cape Town
Processing Record 11 : Gallows Cove
Processing Record 12 : ibis Styles Mt Isa Verona
Processing Record 13 : Hospitality Carnarvon
Processing Record 14 : Alex
Missing result... skipping Bogandinskiy
Processing Record 15 : Residencial Bendiak
Processing Record 16 : Hotel Castelo
Processing Record 17 : Paradise Inn
Processing Record 18 : Port Lincoln Hotel
Processing Record 19 : MacKenzie House Tourist Home B&B
Processing Record 20 : Motel Bela Vista
Processing Record 21 : Playa Canela

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,2,Sālūmbar,24.138922,74.028776,75.33,53,100,1.70,IN,1634669632,Green Building
1,4,Punta Arenas,-53.165952,-70.906213,62.71,51,75,35.68,CL,1634669094,Hotel Hain
2,7,Taltal,-25.408588,-70.484253,63.23,64,27,4.99,CL,1634669634,Alojamiento de Luz y Juan
3,10,Ribeira Grande,38.523672,-28.729263,72.00,68,40,10.36,PT,1634669636,Casa da Japoneira
4,11,Constitución,-35.333303,-72.418517,65.12,59,16,6.33,CL,1634669637,Hotel Playa El Cable
...,...,...,...,...,...,...,...,...,...,...,...
89,514,Biu,10.613732,12.196265,75.99,66,64,2.71,NG,1634670049,EYN LCC MANDARAGIRAU
90,518,Mzimba,-11.900000,33.600000,64.09,65,1,6.24,MW,1634670052,Chipeta Guest House
91,522,Imbituba,-28.246528,-48.666917,65.37,63,100,8.50,BR,1634670055,Apartamentos Universo
92,533,Natal,-5.808014,-35.203046,82.35,63,81,12.59,BR,1634670062,Arituba Park Hotel


In [6]:
#Drop Cities with no hotels 
hotel_df = hotel_df[(hotel_df["Hotel Name"] != "")]
        
hotel_df

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,2,Sālūmbar,24.138922,74.028776,75.33,53,100,1.70,IN,1634669632,Green Building
1,4,Punta Arenas,-53.165952,-70.906213,62.71,51,75,35.68,CL,1634669094,Hotel Hain
2,7,Taltal,-25.408588,-70.484253,63.23,64,27,4.99,CL,1634669634,Alojamiento de Luz y Juan
3,10,Ribeira Grande,38.523672,-28.729263,72.00,68,40,10.36,PT,1634669636,Casa da Japoneira
4,11,Constitución,-35.333303,-72.418517,65.12,59,16,6.33,CL,1634669637,Hotel Playa El Cable
...,...,...,...,...,...,...,...,...,...,...,...
89,514,Biu,10.613732,12.196265,75.99,66,64,2.71,NG,1634670049,EYN LCC MANDARAGIRAU
90,518,Mzimba,-11.900000,33.600000,64.09,65,1,6.24,MW,1634670052,Chipeta Guest House
91,522,Imbituba,-28.246528,-48.666917,65.37,63,100,8.50,BR,1634670055,Apartamentos Universo
92,533,Natal,-5.808014,-35.203046,82.35,63,81,12.59,BR,1634670062,Arituba Park Hotel


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))